# <p align="center">Society of Hispanic Professional Engineers: Financial Mastersheet Logistics</p>
## Functionality: 
- ## Cell 1: This cell iterates over the backend csv to find distinguished committees. This will serve as an input when asked for which committe we are analyzing in Cell 3. 
- ## Cell 2: This cell finds the budget of each semester over an academic year.
- ## Cell 3: This cell compares the budget and actual spending split in semesters. It also highlights bigger purchases. 
- ## Cell 4: This cell creates a bar graph to visually compare the budget and their actual spending.

In [126]:
import matplotlib.pyplot as plt
import pandas as pd
import csv
from dateutil.parser import parse


In [127]:
input_filename = "expenses.csv"

committees = set()  

with open(input_filename, mode='r') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        committees.add(row['Commitee'])

for committee in sorted(committees):  
    print(committee)


Academic Affairs
Alumni
Community Service
Eboard
External
Grad Ambassador
HSO
HSV
Internal
Membershpe
Nationals
Noche De Ciencias
Pen Pals
President
Publicity
RT Fund 23-24
Secretary
Student Advisor
Tech Chair
Treasurer


In [130]:
input_filename = "budgetsheets/shpetina_bs.csv"

df = pd.read_csv(input_filename)

fall_budget = 0
spring_budget = 0

in_fall_events = False
in_spring_events = False

for index, row in df.iterrows():
    event_source = row['Event/Source']
    estimated_expenses = row['Estimated Expenses']
    
    if pd.notna(event_source):
        if 'Fall Events' in event_source:
            in_fall_events = True
            in_spring_events = False
        elif 'Spring Events' in event_source:
            in_fall_events = False
            in_spring_events = True
    
    if in_fall_events and pd.notna(estimated_expenses):
        try:
            fall_budget += float(estimated_expenses.replace('$', '').replace(',', ''))
        except ValueError:
            continue
    
    if in_spring_events and pd.notna(estimated_expenses):
        try:
            spring_budget += float(estimated_expenses.replace('$', '').replace(',', ''))
        except ValueError:
            continue
print(f"Fall Budget: {fall_budget}")
print(f"Spring Budget: {spring_budget}")

Fall Budget: 1185.0
Spring Budget: 3180.0


In [ ]:
import pandas as pd
from dateutil.parser import parse

expenses_filename = "expenses.csv"
budgets_filename = "budgetsheets/shpetina_bs.csv"

expenses_df = pd.read_csv(expenses_filename)
budgets_df = pd.read_csv(budgets_filename)

def parse_date(date_str):
    try:
        return parse(str(date_str)).date()
    except (ValueError, TypeError):
        return None

expenses_df['Date Input'] = expenses_df['Date Input'].apply(parse_date)
expenses_df = expenses_df.dropna(subset=['Date Input'])

expenses_df['Total Cost'] = pd.to_numeric(expenses_df['Total Cost'].astype(str).str.replace('$', '').str.replace(',', ''), errors='coerce').fillna(0)

expenses_df_sorted = expenses_df.sort_values(by='Date Input')

committee_totals = expenses_df_sorted.groupby('Commitee')['Total Cost'].sum().reset_index()
committee_totals.columns = ['Committee', 'Total Cost']

print("Sorted Expenses DataFrame:")
print(expenses_df_sorted)

print("\nTotal Cost for each committee:")
print(committee_totals)

committee_totals.to_csv('committee_totals.csv', index=False)

expenses_df_sorted['Commitee'] = expenses_df_sorted['Commitee'].str.strip().str.lower()

committee = input("Please enter the committee name: ").strip().lower()
committee_expenses = expenses_df_sorted[expenses_df_sorted['Commitee'] == committee]

print(f"\nFiltered expenses for committee '{committee}':")
print(committee_expenses)

fall_expenses = committee_expenses[(committee_expenses['Date Input'].apply(lambda x: x.month in range(8, 12)))]
spring_expenses = committee_expenses[(committee_expenses['Date Input'].apply(lambda x: x.month in range(1, 8)))]

print(f"\nFall expenses for committee '{committee}':")
print(fall_expenses)
print(f"\nSpring expenses for committee '{committee}':")
print(spring_expenses)

total_fall_expenses = fall_expenses['Total Cost'].sum()
total_spring_expenses = spring_expenses['Total Cost'].sum()

fall_difference = fall_budget - total_fall_expenses
spring_difference = spring_budget - total_spring_expenses

fall_status = "Underspent" if fall_difference >= 0 else "Overspent"
spring_status = "Underspent" if spring_difference >= 0 else "Overspent"

fall_large_purchases = fall_expenses[fall_expenses['Total Cost'] > 500]
spring_large_purchases = spring_expenses[spring_expenses['Total Cost'] > 500]

print(f"Committee: {committee}")

print("\nFall Semester:")
print(f"Total Expenses: {total_fall_expenses:.2f}")
print(f"Budget: {fall_budget:.2f}")
print(f"Difference: {fall_difference:.2f} ({fall_status})")
print("Large Purchases:")
for purchase in fall_large_purchases.itertuples():
    print(f"- Description: {purchase.Description}, Total Cost: {purchase._3:.2f}, Comments: {purchase.Comments}")

print("\nSpring Semester:")
print(f"Total Expenses: {total_spring_expenses:.2f}")
print(f"Budget: {spring_budget:.2f}")
print(f"Difference: {spring_difference:.2f} ({spring_status})")
print("Large Purchases:")
for purchase in spring_large_purchases.itertuples():
    print(f"- Description: {purchase.Description}, Total Cost: {purchase._3:.2f}, Comments: {purchase.Comments}")

Sorted Expenses DataFrame:
     Date Input                                        Description  \
0    2023-06-25                                       NILA Summits   
1    2023-07-11                               B2School Fundraiser    
2    2023-08-03                                           RT Merch   
3    2023-08-03                                           RT Merch   
4    2023-08-13                                       Welcome Week   
..          ...                                                ...   
240  2024-05-02                             Senior Gift Embroidery   
241  2024-05-02       30% down payment on white Sox alumni tickets   
242  2024-05-06  Pen Pals: Goodie Bags stuff, paper circuit stu...   
243  2024-05-12                                       SHPE x ALPFA   
244  2024-06-25                          White Sox Alumni Tailgate   

     Total Cost           Commitee  \
0        975.00             Eboard   
1        161.99          Treasurer   
2        866.73   